In [32]:
import pandas as pd
from siuba import *
import numpy as np
from calitp import *

In [33]:
from calitp.storage import get_fs
fs = get_fs()

pd.set_option('display.max_columns', None)

In [34]:
dla = pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx')

In [35]:
dla = to_snakecase(dla)

In [36]:
dt = pd.read_parquet('gs://calitp-analytics-data/data-analyses/grant_misc/BlackCat_TIRCP.parquet')

In [37]:
dla = dla>>select(_.agency_locode, _.agency_name)

In [38]:
dla.head(1)

,agency_locode,agency_name
0,6302,Humboldt Bay Harbor Recreation & Conservation ...


In [39]:
dla = dla.rename(columns={"agency_locode":"locode","agency_name":"name"})

In [40]:
dt

,BlackCat_Orgs,TIRCP_Orgs,BC_TIRCP_merge
0,City of Chowchilla,None,Black Cat Only
1,Madera County,None,Black Cat Only
2,Fresno County Rural Transit Agency,None,Black Cat Only
3,Yuba-Sutter Transit Authority,None,Black Cat Only
4,City of Arcata,None,Black Cat Only
...,...,...,...
235,None,Torrance Transit Department,TIRCP_Only
236,None,Transit Joint Powers Authority of Merced County,TIRCP_Only
237,None,San Francisco Bay Area Water Emergency Transpo...,TIRCP_Only
238,None,City of Sacramento,TIRCP_Only


In [41]:
dt>>filter(_.BC_TIRCP_merge =='Both in TIRCP and BlackCat')

,BlackCat_Orgs,TIRCP_Orgs,BC_TIRCP_merge
8,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,Both in TIRCP and BlackCat
45,Lake Transit Authority,Lake Transit Authority,Both in TIRCP and BlackCat
47,Livermore Amador Valley Transit Authority,Livermore Amador Valley Transit Authority,Both in TIRCP and BlackCat
60,San Joaquin Regional Transit District,San Joaquin Regional Transit District,Both in TIRCP and BlackCat
62,San Mateo County Transit District,San Mateo County Transit District,Both in TIRCP and BlackCat
63,Santa Clara Valley Transportation Authority,Santa Clara Valley Transportation Authority,Both in TIRCP and BlackCat
167,San Francisco Municipal Transportation Agency,San Francisco Municipal Transportation Agency,Both in TIRCP and BlackCat
206,Santa Barbara County Association of Governments,Santa Barbara County Association of Governments,Both in TIRCP and BlackCat
207,Shasta Regional Transportation Agency,Shasta Regional Transportation Agency,Both in TIRCP and BlackCat
210,Antelope Valley Transit Authority,Antelope Valley Transit Authority,Both in TIRCP and BlackCat


In [42]:
dt.head(5)

,BlackCat_Orgs,TIRCP_Orgs,BC_TIRCP_merge
0,City of Chowchilla,None,Black Cat Only
1,Madera County,None,Black Cat Only
2,Fresno County Rural Transit Agency,None,Black Cat Only
3,Yuba-Sutter Transit Authority,None,Black Cat Only
4,City of Arcata,None,Black Cat Only


In [43]:
dt['name']= np.NaN

In [44]:
dt

,BlackCat_Orgs,TIRCP_Orgs,BC_TIRCP_merge,name
0,City of Chowchilla,None,Black Cat Only,NaN
1,Madera County,None,Black Cat Only,NaN
2,Fresno County Rural Transit Agency,None,Black Cat Only,NaN
3,Yuba-Sutter Transit Authority,None,Black Cat Only,NaN
4,City of Arcata,None,Black Cat Only,NaN
...,...,...,...,...
235,None,Torrance Transit Department,TIRCP_Only,NaN
236,None,Transit Joint Powers Authority of Merced County,TIRCP_Only,NaN
237,None,San Francisco Bay Area Water Emergency Transpo...,TIRCP_Only,NaN
238,None,City of Sacramento,TIRCP_Only,NaN


In [45]:
dt.name.fillna(dt['BlackCat_Orgs'], inplace=True)
dt.name.fillna(dt['TIRCP_Orgs'], inplace=True)

In [46]:
dt2 = dt>>select(_.name, _.BC_TIRCP_merge)

In [47]:
dt2

,name,BC_TIRCP_merge
0,City of Chowchilla,Black Cat Only
1,Madera County,Black Cat Only
2,Fresno County Rural Transit Agency,Black Cat Only
3,Yuba-Sutter Transit Authority,Black Cat Only
4,City of Arcata,Black Cat Only
...,...,...
235,Torrance Transit Department,TIRCP_Only
236,Transit Joint Powers Authority of Merced County,TIRCP_Only
237,San Francisco Bay Area Water Emergency Transpo...,TIRCP_Only
238,City of Sacramento,TIRCP_Only


In [48]:
merge = pd.merge(dla, dt2, on="name", how="outer", indicator=True)

In [49]:
merge

,locode,name,BC_TIRCP_merge,_merge
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,NaN,left_only
1,6330.0,Willow Creek Community Services District,NaN,left_only
2,5036.0,Trinidad,NaN,left_only
3,5049.0,Ukiah,NaN,left_only
4,5082.0,Willits,NaN,left_only
...,...,...,...,...
1226,NaN,Santa Monica Big Blue Bus,TIRCP_Only,right_only
1227,NaN,Torrance Transit Department,TIRCP_Only,right_only
1228,NaN,Transit Joint Powers Authority of Merced County,TIRCP_Only,right_only
1229,NaN,San Francisco Bay Area Water Emergency Transpo...,TIRCP_Only,right_only


In [50]:
merge._merge.value_counts()

left_only     991
right_only    190
both           50
Name: _merge, dtype: int64

In [51]:
merge>>filter(_._merge=='both')

,locode,name,BC_TIRCP_merge,_merge
11,6241.0,Mendocino Transit Authority,Black Cat Only,both
42,6428.0,Redwood Coast Transit Authority,Black Cat Only,both
43,6436.0,Lake Transit Authority,Both in TIRCP and BlackCat,both
48,6162.0,Humboldt Transit Authority,Black Cat Only,both
73,6147.0,Plumas County Transportation Commission,Black Cat Only,both
84,6427.0,Lassen Transit Service Agency,Black Cat Only,both
90,6496.0,Shasta Regional Transportation Agency,Both in TIRCP and BlackCat,both
112,6225.0,El Dorado County Transit Authority,Black Cat Only,both
125,6478.0,Tahoe Transportation District,Black Cat Only,both
149,6195.0,Yolo County Transportation District,Black Cat Only,both


In [52]:
(merge>>filter(_._merge=='both')).BC_TIRCP_merge.value_counts()

Black Cat Only                33
Both in TIRCP and BlackCat     9
TIRCP_Only                     8
Name: BC_TIRCP_merge, dtype: int64

In [53]:
(merge>>filter(_._merge=='right_only')).BC_TIRCP_merge.value_counts()

Black Cat Only                168
TIRCP_Only                     21
Both in TIRCP and BlackCat      1
Name: BC_TIRCP_merge, dtype: int64

In [54]:
(merge>>filter(_._merge=='both')).BC_TIRCP_merge.value_counts()

Black Cat Only                33
Both in TIRCP and BlackCat     9
TIRCP_Only                     8
Name: BC_TIRCP_merge, dtype: int64

In [55]:
(merge>>filter(_._merge=='both')).sample(5)

,locode,name,BC_TIRCP_merge,_merge
878,7503.0,San Diego Metropolitan Transit System,Both in TIRCP and BlackCat,both
406,6090.0,Santa Barbara County Association of Governments,Both in TIRCP and BlackCat,both
921,6066.0,San Diego Association of Governments,TIRCP_Only,both
112,6225.0,El Dorado County Transit Authority,Black Cat Only,both
439,6096.0,San Luis Obispo Council of Governments,Black Cat Only,both


In [56]:
def recategorize(df):   
    if (df['_merge']=='right_only') and (df['BC_TIRCP_merge'] == 'Black Cat Only'):
        return 'BlackCat Only'
    elif (df['_merge']=='right_only') and (df['BC_TIRCP_merge'] == 'TIRCP_Only'):
        return 'TIRCP_Only'
    elif (df['_merge']=='right_only') and (df['BC_TIRCP_merge'] == 'Both in TIRCP and BlackCat'):
        return 'TIRCP and BlackCat'
    elif (df['_merge']=='left_only'):
        return 'DLA Only'
    elif (df['_merge']=='both') and (df['BC_TIRCP_merge'] == 'TIRCP Only'):
        return 'TIRCP and DLA'
    elif (df['_merge']=='both') and (df['BC_TIRCP_merge'] == 'Black Cat Only'):
        return 'BlackCat and DLA'
    else: 
        return "TIRCP, BlackCat and DLA"

In [57]:
merge['BC_TIRCP_DLA_merge'] = merge.apply(recategorize, axis = 1)

In [58]:
(merge>>filter(_._merge=='both')).sample(5)

,locode,name,BC_TIRCP_merge,_merge,BC_TIRCP_DLA_merge
406,6090.0,Santa Barbara County Association of Governments,Both in TIRCP and BlackCat,both,"TIRCP, BlackCat and DLA"
257,6510.0,Napa Valley Transportation Authority,Black Cat Only,both,BlackCat and DLA
495,6504.0,Fresno Council of Governments,Black Cat Only,both,BlackCat and DLA
751,6165.0,Riverside Transit Agency,Black Cat Only,both,BlackCat and DLA
750,6507.0,San Bernardino County Transportation Authority,TIRCP_Only,both,"TIRCP, BlackCat and DLA"


In [59]:
(merge>>filter(_._merge=='right_only')).sample(5)

,locode,name,BC_TIRCP_merge,_merge,BC_TIRCP_DLA_merge
1087,NaN,City of Corcoran - Corcoran Area Transit,Black Cat Only,right_only,BlackCat Only
1111,NaN,Friends of Children with Special Needs,Black Cat Only,right_only,BlackCat Only
1123,NaN,Peppermint Ridge,Black Cat Only,right_only,BlackCat Only
1229,NaN,San Francisco Bay Area Water Emergency Transpo...,TIRCP_Only,right_only,TIRCP_Only
1116,NaN,Mental Health Client Action Network,Black Cat Only,right_only,BlackCat Only


In [60]:
merge.drop(columns={'BC_TIRCP_merge','_merge'}, axis=1, inplace=True)

In [61]:
merge

,locode,name,BC_TIRCP_DLA_merge
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,DLA Only
1,6330.0,Willow Creek Community Services District,DLA Only
2,5036.0,Trinidad,DLA Only
3,5049.0,Ukiah,DLA Only
4,5082.0,Willits,DLA Only
...,...,...,...
1226,NaN,Santa Monica Big Blue Bus,TIRCP_Only
1227,NaN,Torrance Transit Department,TIRCP_Only
1228,NaN,Transit Joint Powers Authority of Merced County,TIRCP_Only
1229,NaN,San Francisco Bay Area Water Emergency Transpo...,TIRCP_Only


In [63]:
merge.to_parquet('gs://calitp-analytics-data/data-analyses/grant_misc/BlackCat_TIRCP_DLA.parquet')